In [1]:
import json
import datetime
from datetime import timedelta
import os, zipfile
import pandas as pd
from scipy.optimize import curve_fit
from scipy.stats import chisquare
import numpy as np
import matplotlib.pyplot as plt

from LifeBuddyWebApp import create_app, db
from LifeBuddyWebApp.models import User, Post, Health_description, Health_measure
proxyApp=create_app()
ctx=proxyApp.app_context()
ctx.push()
from sqlalchemy import func

C:\tools\miniconda3\envs\lifebuddy\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [ ]:
#change width of current notebook cells
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
data_zip_med=r"C:\Users\captian2020\Downloads\polar-user-data-export_6103854c-9559-4056-8b5d-89feb507be70.zip"
polar_zip=zipfile.ZipFile(data_zip_med)

#read all files in zip to dictionary of file names and json (each value is a training_session)
polar_data_dict={}
for i in polar_zip.filelist:
    polar_data_dict[i.filename]=json.loads(polar_zip.read(i.filename))

In [48]:
#get max id from Health_description table
max_id=db.session.query(func.max(Health_description.id)).first()[0]

if max_id==None:
    print('cest none')
    

cest none


In [14]:
df=pd.DataFrame()

In [15]:
df

""


In [16]:
df['new_coloumn2']=[1,2]

In [17]:
df

,new_coloumn2
0,1
1,2


In [49]:
#read all training sessions into df's inside of a dict
polar_df_dict1={}
polar_df_dict2={}
for i,j in polar_data_dict.items():
    if 'training-session-' in i:
        #Make Health_description df
        if max_id==None:
            max_id=1
        else:
            max_id +=1
        df1=pd.DataFrame()
        df1['id']=[max_id]
        df1['var_activity']=[j['name']]
        df1['var_type']=['heart rate']
        df1['var_periodicity']=['seconds']
        df1['var_unit']=['heart rate per second']
        df1['var_timezone_utc_delta_in_mins']=[j['timeZoneOffset']]
        df1['time_stamp_utc']=[datetime.datetime.utcnow()]
        df1['user_id']=[1]#replace with current user
        df1['source_filename']=[i]
        polar_df_dict1[i]=df1
        
        var_datetime_utc=j['timeZoneOffset']
        var_datetime_utc_list=[datetime.datetime.strptime(
            k['dateTime']  ,'%Y-%m-%dT%H:%M:%S.%f') + timedelta(
                minutes=var_datetime_utc) for k in j['exercises'][0]['samples']['heartRate']]
        var_values_list=[k['value'] for k in j['exercises'][0]['samples']['heartRate']]
        df2=pd.DataFrame(list(zip(var_datetime_utc_list,var_values_list)), columns=[
            'var_datetime_utc','var_value'])
        df2['description_id']=max_id
        polar_df_dict2[i]=df2

In [51]:
polar_df_dict2[list(polar_df_dict2.keys())[1]]

,var_datetime_utc,var_value,description_id
0,2021-06-07 07:54:36.148,171,2
1,2021-06-07 07:54:37.148,171,2
2,2021-06-07 07:54:38.148,171,2
3,2021-06-07 07:54:39.148,171,2
4,2021-06-07 07:54:40.148,171,2
...,...,...,...
245,2021-06-07 07:58:41.148,113,2
246,2021-06-07 07:58:42.148,113,2
247,2021-06-07 07:58:43.148,113,2
248,2021-06-07 07:58:44.148,113,2
